In [3]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm

from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

RANDOM_SEED = 777
np.random.seed(RANDOM_SEED)
np.random.SeedSequence(RANDOM_SEED)
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)

In [4]:
metadata = pd.read_csv('../input/is-this-crazy/metadata.csv')
sup_metadata = pd.read_csv('../input/is-this-crazy/supplemental_metadata.csv')
train_labels = pd.read_csv('../input/is-this-crazy/train_labels.csv', index_col="sample_id")
val_labels = pd.read_csv('../input/is-this-crazy/val_labels.csv',index_col='sample_id')
# targets = pd.read_csv('targets.csv',index_col='sample_id')
submission_format = pd.read_csv('../input/is-this-crazy/submission_format.csv')

In [5]:
targets = pd.concat([train_labels,val_labels],axis=0)
targets.shape

(1059, 10)

In [6]:
sam_id = metadata[metadata.instrument_type == 'sam_testbed'].index

In [7]:
# only change the data sets here...
train_path = '../input/ms-gen-train-test-data/train_temp_50.csv'
test_path =  '../input/ms-gen-train-test-data/test_temp_50.csv'
val_path = '../input/ms-gen-train-test-data/val_temp_50.csv'

In [8]:
oof_train_path = 'oof_train_temp_50.csv'
oof_test_path = 'oof_test_temp_50.csv'

In [9]:
%%time
test_100 = pd.read_csv(test_path,index_col=0,header=[0,1])
train_100 = pd.read_csv(train_path,index_col=0,header=[0,1])
val_100 = pd.read_csv(val_path,index_col=0,header=[0,1])

val_100.index = metadata[metadata.split == 'val'].sample_id

val_100.sort_index(inplace=True,axis=1)
test_100.sort_index(inplace=True,axis=1)
train_100.sort_index(inplace=True,axis=1)

CPU times: user 2.24 s, sys: 182 ms, total: 2.43 s
Wall time: 3.46 s


In [10]:
# train_100.drop(columns=[('sample_id_encoded', 'Unnamed: 1601_level_1')],axis=1,inplace=True)

In [11]:
train_set_100 = pd.concat([train_100,val_100],axis=0)
test_set_100 = pd.concat([val_100,test_100],axis=0)
train_set_100.shape, test_set_100.shape

((1059, 4000), (804, 4000))

In [12]:
skf = StratifiedKFold(n_splits=20,random_state=RANDOM_SEED,shuffle=True)
log_loss_scorer = make_scorer(log_loss,needs_proba=True)

In [13]:
phy = 'phyllosilicate'
bas = 'basalt'
lr = LogisticRegression(solver='liblinear',C=10,random_state=RANDOM_SEED,penalty='l1')
lr

LogisticRegression(C=10, penalty='l1', random_state=777, solver='liblinear')

In [14]:
def get_coefs(X,y,c):
    lr = LogisticRegression(solver='liblinear',C=c,random_state=RANDOM_SEED,penalty='l1')
    print(cross_val_score(lr,X,y,cv=skf,scoring=log_loss_scorer).mean())
    lr.fit(X,y)
    # print(log_loss(y,lr.predict_proba(X)))
    coefs =  lr.coef_
    coefs_list  = []
    for id,wt in enumerate(tqdm(coefs[0])):
        if wt != 0:
            coefs_list.append(id)
    print(len(coefs_list))
    return coefs_list

In [15]:
def get_mz(mz,df):
    ''' returns list of columns for given mz'''
    cols_list = []
    for col in df.columns:
        if col[0] == str(mz):
            cols_list.append(col)
    return cols_list

In [16]:
def get_total_abundance(df):
    ''' returns total abundance for each sample'''
    tot_abund_df = pd.DataFrame(index=df.index)
    tot_abund_df['tot_abund'] = df.sum(axis=1)
    for mz in tqdm(range(0,100)):
        tot_abund_df[f"tot_abund_{mz}"] = df[get_mz(mz,df)].sum(axis=1)
    print(tot_abund_df.shape)
    return tot_abund_df

In [17]:
def get_grads(df,bin_size=5):
    ''' returns gradients for each mz'''
    grad_df = pd.DataFrame(index=df.index)
    for mz in tqdm(range(0,100)):
        df_mz = df.loc[:,get_mz(mz,df)]
        # df_mz_col1 = df_mz.iloc[:,0].values
        df_mz = df_mz.diff(axis=1)
        # df_mz.iloc[:,0] = df_mz_col1
        df_mz.dropna(axis='columns',inplace=True)
        # df_mz = df_mz/bin_size
        df_mz.columns = [col[0]+ '_' + col[1] + '_' + 'grad' for col in df_mz.columns]
        grad_df = pd.concat([grad_df,df_mz],axis=1)
    print(grad_df.shape)

    return  grad_df

In [18]:
def get_mean_std(df):
    ''' returns mean and std for each mz'''
    mean_std_df = pd.DataFrame(index=df.index)
    for mz in tqdm(range(0,100)):
        df_mz = df.loc[:,get_mz(mz,df)]
        df_mz[f'{mz}_mean'] = df_mz.mean(axis=1)
        df_mz[f'{mz}_std'] = df_mz.std(axis=1)
        mean_std_df = pd.concat([mean_std_df,df_mz],axis=1)
    print(mean_std_df.shape)
    return mean_std_df

In [19]:
# def add_fts(df):
#     tot_abund_df = get_total_abundance(df)
#     grad_df = get_grads(df)
#     mean_std_df = get_mean_std(df)
#     new_df = pd.concat([tot_abund_df,grad_df,mean_std_df],axis=1)
#     return new_df

In [20]:
train_set_100.shape, test_set_100.shape

((1059, 4000), (804, 4000))

In [21]:
assert (train_set_100.columns == test_set_100.columns).all()

In [22]:
t_100 = get_total_abundance(train_set_100)
g_100 = get_grads(train_set_100)
m_100 = get_mean_std(train_set_100)

100%|██████████| 100/100 [00:00<00:00, 184.74it/s]


(1059, 101)


100%|██████████| 100/100 [00:01<00:00, 64.72it/s]


(1059, 3900)


100%|██████████| 100/100 [00:01<00:00, 68.97it/s]

(1059, 4200)


In [23]:
t_100_test = get_total_abundance(test_set_100)
g_100_test = get_grads(test_set_100)
m_100_test = get_mean_std(test_set_100)


100%|██████████| 100/100 [00:00<00:00, 185.64it/s]


(804, 101)


100%|██████████| 100/100 [00:01<00:00, 70.49it/s]


(804, 3900)


100%|██████████| 100/100 [00:01<00:00, 77.07it/s]

(804, 4200)


In [24]:
t_100_test.sort_index(inplace=True,axis=1)
t_100.sort_index(inplace=True,axis=1)

m_100_test.sort_index(inplace=True,axis=1)
m_100.sort_index(inplace=True,axis=1)

g_100_test.sort_index(inplace=True,axis=1)
g_100.sort_index(inplace=True,axis=1)

In [25]:
assert (t_100.columns == t_100_test.columns).all()
assert (g_100.columns == g_100_test.columns).all()
assert (m_100.columns == m_100_test.columns).all()

In [26]:
ntrain = 1059
ntest = 804
NFOLDS = 20

In [27]:
oof_train_df = pd.DataFrame(index=metadata[metadata.split != 'test'].index, columns=['sample_id',*targets.columns])
oof_test_df = pd.DataFrame(index=metadata[metadata.split != 'train'].index, columns=['sample_id',*targets.columns])

In [28]:
agg_loss = []
for target in targets.columns:
    print('TARGET ',target)
    imp_fts = {}
    
    #  get all imp coefs for the target
    t_100_coefs = get_coefs(t_100,targets[target],5)
    g_100_coefs = get_coefs(g_100,targets[target],5)
    m_100_coefs = get_coefs(m_100,targets[target],10)
    o_100_coefs = get_coefs(train_set_100,targets[target],10)
    
    imp_fts['o_100_coefs'] = o_100_coefs
    imp_fts['t_100_coefs'] = t_100_coefs
    imp_fts['g_100_coefs'] = g_100_coefs
    imp_fts['m_100_coefs'] = m_100_coefs

    # get all imp_dfs and concat them
    train_imp_ = train_set_100.iloc[:,o_100_coefs]
    t_100_imp_ = t_100.iloc[:,t_100_coefs]
    g_100_imp_ = g_100.iloc[:,g_100_coefs]
    m_100_imp_ = m_100.iloc[:,m_100_coefs]

    print(train_imp_.shape,t_100_imp_.shape,g_100_imp_.shape,m_100_imp_.shape)
    X = pd.concat([train_imp_,t_100_imp_,g_100_imp_,m_100_imp_],axis=1)
    print(X.shape)

    # same goes for test set
    test_imp_ = test_set_100.iloc[:,o_100_coefs]
    t_100_imp_test = t_100_test.iloc[:,t_100_coefs]
    g_100_imp_test = g_100_test.iloc[:,g_100_coefs]
    m_100_imp_test = m_100_test.iloc[:,m_100_coefs]

    print(test_imp_.shape,t_100_imp_test.shape,g_100_imp_test.shape,m_100_imp_test.shape)
    
    X_test = pd.concat([test_imp_, t_100_imp_test, g_100_imp_test, m_100_imp_test],axis=1)
    print(X_test.shape)
    
    #  add feature
    X['is_sam'] = 0
    X.loc[sam_id[:12],'is_sam'] = 1
    X_test['is_sam'] = 0
    X_test.loc[sam_id[12:],'is_sam'] = 1
    
    
    assert (X.columns == X_test.columns).all()

    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    Y = targets[target]

    cb_clf = CatBoostClassifier(verbose=999,random_seed=RANDOM_SEED,n_estimators=5000)
    cv_loss = []
    for fold, (train_id, test_id) in enumerate(tqdm(skf.split(X,Y))):

        X_train, Y_train = X.iloc[train_id,], Y.iloc[train_id,]
        X_val, Y_val = X.iloc[test_id,], Y.iloc[test_id,]
        #  ohh man unwanted semi colon in iloc throws an error...
        
        # train model  and pred on oof folds
        cb_clf.fit(X_train.values,Y_train)
        y_preds = cb_clf.predict_proba(X_val.values)[:,1]
        fold_loss = log_loss(Y_val,y_preds)
        print(f'fold Loss {fold}: ',fold_loss)
        cv_loss.append(fold_loss)

        oof_train[test_id] = y_preds
        oof_test_skf[fold,:] = cb_clf.predict_proba(X_test.values)[:,1]
        
        print("---------------------------------------------------------------------------")
        
    print('Training Loss:', log_loss(Y,cb_clf.predict_proba(X.values)[:,1]))
    
    oof_test[:] = oof_test_skf.mean(axis=0)
    oof_test_df[target] = oof_test
    oof_train_df[target] = oof_train

    print(f'Loss for {target}:')
    print(np.array(cv_loss).mean(),np.array(cv_loss).std())
    agg_loss.append(np.array(cv_loss).mean())
    print("==================================================================================================")

print('AGG Loss:',np.array(agg_loss))
print('AGG Loss mean:',np.array(agg_loss).mean())

TARGET  basalt
0.3006324473745309


100%|██████████| 101/101 [00:00<00:00, 442196.98it/s]


26
0.2578836551987977


100%|██████████| 3900/3900 [00:00<00:00, 866775.41it/s]


67
0.2561338693625336


...

4995:	learn: 0.0042607	total: 1m 58s	remaining: 94.9ms
4999:	learn: 0.0042576	total: 1m 58s	remaining: 0us
fold Loss 17:  0.09228773018948788
---------------------------------------------------------------------------
Learning rate set to 0.002361
0:	learn: 0.6891750	total: 39ms	remaining: 3m 14s
999:	learn: 0.0342986	total: 23.9s	remaining: 1m 35s
1998:	learn: 0.0141045	total: 47.5s	remaining: 1m 11s
2997:	learn: 0.0081159	total: 1m 11s	remaining: 47.4s
3996:	learn: 0.0053336	total: 1m 34s	remaining: 23.8s


19it [37:41, 119.33s/it]

4995:	learn: 0.0040676	total: 1m 58s	remaining: 94.9ms
4999:	learn: 0.0040642	total: 1m 58s	remaining: 0us
fold Loss 18:  0.0698168406158262
---------------------------------------------------------------------------
Learning rate set to 0.002362
0:	learn: 0.6885174	total: 81.5ms	remaining: 6m 47s
999:	learn: 0.0323757	total: 23.9s	remaining: 1m 35s
1998:	learn: 0.0134112	total: 47.5s	remaining: 1m 11s
2997:	learn: 0.0078002	total: 1m 11s	remaining: 47.8s
3996:	learn: 0.0051407	total: 1m 34s	remaining: 23.8s


20it [39:41, 119.09s/it]

4995:	learn: 0.0036371	total: 1m 59s	remaining: 95.3ms
4999:	learn: 0.0036329	total: 1m 59s	remaining: 0us
fold Loss 19:  0.11651954450646332
---------------------------------------------------------------------------
Training Loss: 0.009180927525886742
Loss for sulfide:
0.0682594572649331 0.05358530790848837
AGG Loss: [0.13863498 0.09386363 0.15080343 0.20577805 0.01561893 0.13829565
 0.19726156 0.17902981 0.16177024 0.06825946]
AGG Loss mean: 0.13493157515767454


In [29]:
print(np.array(agg_loss).mean())

0.13493157515767454


In [30]:
test_preds = oof_test_df.iloc[:293]
train_preds = oof_train_df.iloc[766:]

In [31]:
display(test_preds)
display(train_preds)

,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
766,NaN,0.003087,0.002623,0.008387,0.006500,0.000106,0.007050,0.007246,0.090297,0.003326,0.000508
767,NaN,0.038983,0.009572,0.001873,0.016731,0.000037,0.007850,0.012565,0.087384,0.014951,0.001254
768,NaN,0.873906,0.085162,0.001803,0.032249,0.000226,0.011131,0.839002,0.894410,0.025066,0.000528
769,NaN,0.011848,0.003262,0.028379,0.022904,0.000049,0.964711,0.012030,0.005945,0.961436,0.000794
770,NaN,0.007110,0.004781,0.013356,0.731771,0.000057,0.994588,0.922567,0.006387,0.008056,0.000322
...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,0.006871,0.002659,0.030218,0.008685,0.000046,0.008368,0.018227,0.066457,0.013813,0.000289
1055,NaN,0.007367,0.011137,0.014074,0.049163,0.000195,0.014185,0.020265,0.828172,0.002057,0.000769
1056,NaN,0.004260,0.008478,0.002650,0.010917,0.998040,0.002441,0.005689,0.005216,0.003766,0.000438
1057,NaN,0.006734,0.936341,0.007942,0.060972,0.000102,0.007761,0.041495,0.010255,0.004341,0.000585


,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
766,NaN,0.003265,0.003013,0.014348,0.006007,0.000125,0.008846,0.009950,0.253597,0.003806,0.000380
767,NaN,0.066180,0.004321,0.001953,0.021002,0.000038,0.010305,0.024050,0.262126,0.020939,0.002674
768,NaN,0.429562,0.687648,0.002164,0.049875,0.000285,0.010401,0.489729,0.696324,0.074093,0.000445
769,NaN,0.018502,0.003060,0.068667,0.030043,0.000050,0.893590,0.013359,0.007411,0.904344,0.000805
770,NaN,0.008069,0.006281,0.011712,0.150796,0.000064,0.994470,0.818067,0.006612,0.010538,0.000296
...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,0.008966,0.001512,0.032048,0.011596,0.000051,0.009910,0.031019,0.145185,0.024195,0.000317
1055,NaN,0.010221,0.011941,0.015794,0.063976,0.000261,0.020876,0.037727,0.368779,0.002424,0.000679
1056,NaN,0.004745,0.013683,0.002780,0.011580,0.997237,0.002587,0.005992,0.005848,0.004506,0.000473
1057,NaN,0.008121,0.835721,0.008161,0.117901,0.000137,0.012380,0.078760,0.011798,0.006567,0.000663


In [32]:
agg_loss_train = []
for tar in targets.columns:
    loss = log_loss(val_labels[tar],train_preds[tar])
    print(tar, loss)
    agg_loss_train.append(loss)
print(np.array(agg_loss_train).mean())

basalt 0.14449489986316724
carbonate 0.08418013184828872
chloride 0.1667769269696335
iron_oxide 0.24588512671700838
oxalate 0.0008297396081711584
oxychlorine 0.13437332481189435
phyllosilicate 0.17696919781158027
silicate 0.15742172400152138
sulfate 0.18366673539758843
sulfide 0.05790219780984483
0.13525000048386984


In [33]:
agg_loss_test = []
for tar in targets.columns:
    loss = log_loss(val_labels[tar],test_preds[tar])
    print(tar, loss)
    agg_loss_test.append(loss)
print(np.array(agg_loss_test).mean())

basalt 0.027110592816784594
carbonate 0.015878631081548456
chloride 0.029103966080426136
iron_oxide 0.05811756211872979
oxalate 0.000444174440591192
oxychlorine 0.026348857501253188
phyllosilicate 0.04256489411299977
silicate 0.037714939791791996
sulfate 0.025998058857335582
sulfide 0.004889064357077933
0.02681707411585386


In [34]:
oof_test_df.sample_id = metadata[metadata.split != 'train'].sample_id
oof_train_df.sample_id = metadata[metadata.split != 'test'].sample_id

In [35]:
oof_train_df.to_csv(oof_train_path, index=False)
oof_test_df.to_csv(oof_test_path, index=False)